In [ ]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install transformers datasets accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Sun May 28 07:40:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 抓dataset

In [ ]:
from datasets import list_datasets, load_dataset
from pprint import pprint
import json

In [ ]:
with open("train_social2.json",'r',encoding='utf-8') as train:
    data = json.load(train)
    train_sen1 = []
    trainlabel = []
    for line in data:
        train_sen1.append(line[0]+"</s>"+line[1])
        trainlabel.append(line[2])
#pprint(data)


In [ ]:
with open("dev_social2.json",'r',encoding='utf-8') as dev:
    data = json.load(dev)
    eval_sen1 = []
    eval_sen2 = []
    evallabel = []
    for line in data:
        eval_sen1.append(line[0]+"</s>"+line[1])
        evallabel.append(line[2])
#pprint(data)


In [ ]:
with open("test_social2.json",'r',encoding='utf-8') as test:
    data = json.load(test)
    test_sen1 = []
    test_sen2 = []
    testlabel = []
    for line in data:
        test_sen1.append(line[0]+"</s>"+line[1])
        testlabel.append(line[2])
#pprint(data)


## dataset裡分成三個 train(訓練用) validation(每次訓縣完測試訓練情形用) test(最終算分用)

## 把每一筆資料集的每一項拆開

# 選擇和使用tokenizer，具體要使用哪個tokenizer可以去huggingface官網找

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#from transformers import AutoTokenizer,BertTokenizerFast, BertTokenizer, AdamW, BertForQuestionAnswering
#tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")懶人用法
#tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
#tokenizer_fast = BertTokenizerFast.from_pretrained("bert-base-uncased")
#from transformers import BertTokenizer, BertForSequenceClassification

#tokenizer = BertTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
tokenizer

T5TokenizerFast(name_or_path='google/mt5-small', vocab_size=250100, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True)

In [ ]:
train_encodings = tokenizer(train_sen1, train_sen2, truncation=True, padding=True)
print(train_encodings)
eval_encodings = tokenizer(eval_sen1, eval_sen2, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
#print(tokenizer.decode(train_encodings['input_ids'][0]))
#print(train_encodings['input_ids'][0])
#print("token_type_ids\n", train_encodings['token_type_ids'][0])
#print("attention_mask\n", train_encodings['attention_mask'][0])

#print(len(train_encodings['input_ids'][0]))
#print(len(train_encodings['token_type_ids'][0]))

## 加入label(答案)

In [ ]:
def add_targets(encodings,label):
    encodings.update({'labels':label})
add_targets(train_encodings,trainlabel)
add_targets(eval_encodings,evallabel)


In [ ]:
print(train_encodings.keys())
print(tokenizer.decode(train_encodings['labels']))

dict_keys(['input_ids', 'attention_mask', 'labels'])
</s><pad><pad></s></s></s></s></s><pad></s></s></s></s><pad><pad></s><pad><pad><pad></s></s><pad></s></s><pad></s></s></s></s></s><pad><pad></s><pad></s><pad><pad></s></s></s></s><pad></s></s></s></s></s><pad></s><pad><pad></s><pad><pad></s><pad></s><pad><pad></s><pad></s></s></s></s><pad><pad></s><pad></s></s><pad></s></s></s><pad><pad><pad></s><pad></s></s><pad></s></s><pad><pad></s><pad><pad></s><pad><pad></s></s></s></s></s></s><pad></s></s><pad></s><pad></s></s><pad></s><pad><pad><pad></s></s><pad></s><pad><pad></s></s><pad></s><pad></s></s><pad><pad></s></s></s><pad><pad></s><pad><pad></s><pad><pad></s><pad></s></s><pad><pad></s></s><pad><pad></s><pad></s></s><pad><pad><pad></s></s><pad></s></s><pad></s></s></s></s><pad></s></s><pad></s></s><pad><pad></s><pad></s></s></s><pad></s></s><pad></s></s></s><pad><pad></s><pad></s><pad><pad></s></s><pad><pad></s><pad><pad><pad></s></s><pad><pad><pad></s><pad><pad><pad><pad></s></s><pad

# 定義dataset 並轉換成tensor格式


In [ ]:

from torch.utils import data
import torch

class Dataset(torch.utils.data.Dataset):
  def __init__(self, encodings):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(eval[idx]) for key, eval in self.encodings.items()}

  def __len__(self):
    return len(self.encodings.input_ids)

In [ ]:
train_dataset = Dataset(train_encodings)
eval_dataset = Dataset(eval_encodings)

In [ ]:
train_dataset[0]

{'input_ids': tensor([  259, 32030, 17225,  ...,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]),
 'labels': tensor(1)}

# 載入模型架構

In [ ]:
from transformers import MT5Config,MT5Model,MT5ForConditionalGeneration,AutoModelForSeq2SeqLM
config = MT5Config.from_pretrained('google/mt5-small', num_labels=2)  #num_labels 設定類別數
model =  MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

In [ ]:
print(model)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

## 該來訓練模型囉

In [ ]:
#import gc
#torch.cuda.empty_cache()
#gc.collect()

In [ ]:
#!ps -aux|grep python

In [ ]:
#!kill -9 8885

In [ ]:
import logging
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader
from tqdm.auto import tqdm, trange
import math

import transformers
from accelerate import Accelerator
from transformers import (
    AdamW,
    AutoConfig,
    default_data_collator,
    get_scheduler
)

train_batch_size = 4      # 設定 training batch size 
eval_batch_size = 4      # 設定 eval batch size
num_train_epochs = 1      # 設定 epoch 

In [ ]:
data_collator = default_data_collator
train_dataloader = DataLoader(train_dataset, shuffle=True, collate_fn=data_collator, batch_size=train_batch_size)
eval_dataloader = DataLoader(eval_dataset, collate_fn=data_collator, batch_size=eval_batch_size)

In [ ]:
learning_rate=3e-5          # 設定 learning_rate
gradient_accumulation_steps = 1   # 設定 幾步後進行反向傳播

no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },                                
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
max_train_steps = num_train_epochs * num_update_steps_per_epoch
print('max_train_steps', max_train_steps)

# scheduler
lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=max_train_steps,
)

max_train_steps 2807


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

eval_dataloader
metric = load_metric("accuracy")

<ipython-input-29-570f2d83a7b4>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


真正開始訓練

In [ ]:
logger = logging.getLogger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state)
output_dir = '/content/drive2/Shareddrives'  # your folder


total_batch_size = train_batch_size * accelerator.num_processes * gradient_accumulation_steps

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_train_epochs}")
logger.info(f"  Instantaneous batch size per device = {train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Gradient Accumulation steps = {gradient_accumulation_steps}")
logger.info(f"  Total optimization steps = {max_train_steps}")


completed_steps = 0
best_epoch = {"epoch": 0, "acc": 0 }

for epoch in trange(num_train_epochs, desc="Epoch"):#trange是print進度條的方式

  model.train()
  for step, batch in enumerate(tqdm(train_dataloader, desc="Iteration")):
    print(batch)
    for i in batch:
        print(batch[i])
        for j in batch[i]:
            print(j)
    outputs = model(**batch)
    print(outputs)
    #loss = outputs
    loss = outputs.loss
    #loss = loss / gradient_accumulation_steps
    accelerator.backward(loss)
    #if step % gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1: 把if刪掉了
    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    completed_steps += 1

    if step % 50 == 0:
      print({'epoch': epoch, 'step': step, 'loss': loss.item()})

    if completed_steps >= max_train_steps:
      break
      
  logger.info("***** Running eval *****")
  model.eval()
  for step, batch in enumerate(tqdm(eval_dataloader, desc="eval Iteration")):
    outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    metric.add_batch(
        predictions=accelerator.gather(predictions),
        references=accelerator.gather(batch["labels"]),
    )

  eval_metric = metric.compute()
  logger.info(f"epoch {epoch}: {eval_metric}")
  if eval_metric['accuracy'] > best_epoch['acc']:
    best_epoch.update({"epoch": epoch, "acc": eval_metric['accuracy']})

  if output_dir is not None:
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir + '/' + 'epoch_' + str(epoch), save_function=accelerator.save)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2807 [00:00<?, ?it/s]

{'input_ids': tensor([[   259,  24636,  24696,  ...,      0,      0,      0],
        [   259, 108517, 108517,  ...,      0,      0,      0]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'), 'labels': tensor([1, 1], device='cuda:0')}
tensor([[   259,  24636,  24696,  ...,      0,      0,      0],
        [   259, 108517, 108517,  ...,      0,      0,      0]],
       device='cuda:0')
tensor([  259, 24636, 24696,  ...,     0,     0,     0], device='cuda:0')
tensor([   259, 108517, 108517,  ...,      0,      0,      0], device='cuda:0')
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0')
tensor([1, 1, 1,  ..., 0, 0, 0], device='cuda:0')
tensor([1, 1, 1,  ..., 0, 0, 0], device='cuda:0')
tensor([1, 1], device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 25>:34                                                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/mt5/modeling_mt5.py:1748 in forward  │
│                                                                                                  │
│   1745 │   │   │   │   decoder_attention_mask = decoder_attention_mask.to(self.decoder.first_de  │
│   1746 │   │                                                                                     │
│   1747 │   │   # Decode                                                                          │
│ ❱ 1748 │   │   decoder_outputs = self.decoder(                                                   │
│   1749 │   │   │   input_ids=decoder_input_ids,                                                  │
│   1750 │   │   │   attention_mask=decoder_attention_mask,                                        │
│   1751 │   │   │   inputs_embeds=decoder_inputs_embeds,                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501 in _call_impl            │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/models/mt5/modeling_mt5.py:958 in forward   │
│                                                                                                  │
│    955 │   │   │   assert self.embed_tokens is not None, "You have to initialize the model with  │
│    956 │   │   │   inputs_embeds = self.embed_tokens(input_ids)                                  │
│    957 │   │                                                                                     │
│ ❱  958 │   │   batch_size, seq_length = input_shape                                              │
│    959 │   │                                               

## 最好的某次訓練成果

In [ ]:
print(best_epoch)

# 訓練成果驗證

In [ ]:
from transformers import T5TokenizerFast, MT5Config, MT5ModelForSeq2SeqLM, default_data_collator
from torch.utils.data import DataLoader
from accelerate import Accelerator
from tqdm.auto import tqdm

In [ ]:
#cd drive

In [ ]:
#ls

In [ ]:
config = MT5Config.from_pretrained("./drive2/Shareddrives/epoch_2/config.json") 
model = MT5ModelForSeq2SeqLM.from_pretrained("./drive2/Shareddrives/epoch_2/pytorch_model.bin", config = config).to(device)

In [ ]:
def mrpc_model(model, sen1, sen2):
  input_encodings = tokenizer([sen1], [sen2], padding='max_length', truncation=True)
  input_dataset = Dataset(input_encodings)
  #print(input_encodings)
  #print(input_dataset[0])
  data_collator = default_data_collator
  input_dataloader = DataLoader(input_dataset, collate_fn=data_collator, batch_size=1)  

  accelerator = Accelerator()
  model, input_dataloader = accelerator.prepare(model, input_dataloader)

  for batch in input_dataloader:
    outputs = model(**batch)
    predicted = outputs.logits.argmax(dim=-1)
  return predicted

### 可以拿來玩的地方

In [ ]:
sen1="lisa goes to school everyday"
sen2="lisa everyday goes to school"
#sen1="lisa is a singer"
#sen2="lisa is not a singer"

predict = mrpc_model(model, sen1, sen2)
print("sentence= : ", sen1)
print("sentence= : ", sen2)

print("predict_label : ", predict.item())
if predict.item():
  print("有關聯")
else:
  print("沒關聯")


In [ ]:
cnt=0
errorcnt=0
answer=[]
for i in range(len(testlabel)):
    if testlabel[i]==0 or testlabel[i]==1:
        cnt+=1
        sen1=test_sen1[i]
        sen2=test_sen2[i]
        predict=mrpc_model(model,sen1,sen2)
        #print("sentence= : ", sen1)
        #print("sentence= : ", sen2)
        answer.append(predict.item())
        #print("predict_label : ", predict.item())
        if predict.item()!=testlabel[i]:
            errorcnt+=1
        
print(cnt)
print(errorcnt)
print(1-(errorcnt/cnt))

結果存進google drive

In [ ]:
from sklearn.metrics import classification_report
print("no support=0. support=1")
print(classification_report(testlabel,answer))

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
cd gdrive/MyDrive/Colab Notebooks

In [ ]:

#torch.save(model,"./test_model2")
torch.save(model,"./test_model2.bin")